In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data
df=pd.read_csv('df_processed.csv')
df_testids=pd.read_csv('data/Test_IDs.csv')
df_sta=pd.read_csv('data/status.csv')
df_nan_map=pd.read_csv('df_nan_map.csv')
df=df.set_index('Customer ID')
df_nan_map=df_nan_map.set_index('Customer ID')

In [3]:
df_sta['Churn Category']=df_sta['Churn Category'].map({'No Churn':0,'Competitor':1,'Dissatisfaction':2,'Attitude':3,'Price':4,'Other':5})
df_train=df.reindex(df_sta['Customer ID'])
df_nan_map=df_nan_map.reindex(df_sta['Customer ID'])
df_sta=df_sta.set_index('Customer ID')
# df_nan_map


In [4]:
# thres=20 #number of nan
# filter=df_nan_map.sum(axis=1)<thres
# print(df_sta.shape)
# df_train=df_train.loc[filter,:]
# df_sta=df_sta.loc[filter,:]
# print(df_sta.shape)

In [5]:
y=df_sta.to_numpy().ravel()
X=df_train.to_numpy()
print(X.shape)

(4226, 43)


In [6]:
# from sklearn import preprocessing
# X_train=preprocessing.scale(X_train)

In [7]:
# from sklearn.model_selection import train_test_split
# from sklearn import svm
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=0)
# clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
# clf.score(X_val, y_val)

In [8]:
#using random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer,f1_score

clf = RandomForestClassifier(n_estimators = 10, random_state = 0,n_jobs=-1)
#pipe= Pipeline([ ('scaler',StandardScaler()),('rf',RandomForestClassifier(n_estimators = 1000, random_state = 0,n_jobs=-1) )])
scores = cross_val_score(clf, X, y, cv=5,scoring=make_scorer(f1_score, average='macro'))
scores.mean()


0.29072068101438775

In [9]:
#using logistic regession
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#clf = LogisticRegression(random_state=42,n_jobs=-1,multi_class='ovr',C=0.5).fit(X_train, y_train)
pipe= Pipeline([('scaler',StandardScaler()),('clf', LogisticRegression(random_state=42,n_jobs=-1,multi_class='ovr',C=0.5,class_weight='balanced'))])
scores = cross_val_score(pipe, X, y, cv=5,scoring=make_scorer(f1_score, average='macro'))
scores.mean()

0.3098954071039285

In [10]:
#using svm linear
# from sklearn.svm import SVC
# pipe= Pipeline([('scaler',StandardScaler()),('clf',SVC())])
# scores = cross_val_score(pipe, X, y, cv=5,scoring=make_scorer(f1_score, average='macro'))
# scores.mean()

from sklearn.svm import SVC
pipe= Pipeline([('scaler',StandardScaler()),('clf',SVC(kernel='rbf',gamma='auto',class_weight='balanced'))])
scores = cross_val_score(pipe, X, y, cv=5,scoring=make_scorer(f1_score, average='macro'))
scores.mean()

#0.31904

0.3062782761538274

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  

clf = MLPClassifier( solver = 'lbfgs',# 'adam', # 'lbfgs', 
                     alpha = 1e-5,
                     hidden_layer_sizes=(43, 5),
                     activation ='relu', # 
                     learning_rate='invscaling', # Only used when solver='sgd'
                     random_state = 42, 
                     shuffle = True,
                     warm_start=True,
                     max_iter = 2000)

pipe= Pipeline([('scaler', StandardScaler()), ('clf', clf)])
scores = cross_val_score(pipe, X, y, cv=5, scoring=make_scorer(f1_score, average='macro'))
scores.mean()

# 0.29182652885060895 (43, 5) lbfgs # Still can't converage

/home/spiderkiller/miniconda3/envs/html/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [12]:
'''
#validation(SVM)
from sklearn.svm import SVC
#best gamma
gamma_list=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
scores_list=[]
best_pipe=pipe
for gamma in gamma_list:
    print(gamma)
    pipe= Pipeline([('scaler',StandardScaler()),('clf',SVC(kernel='rbf',gamma=gamma,class_weight='balanced'))])
    scores = cross_val_score(pipe, X, y, cv=5,scoring=make_scorer(f1_score, average='macro'))
    if all(scores.mean()>x for x in scores_list):
        best_pipe=pipe
    scores_list.append(scores.mean())
scores_list
'''

"\n#validation(SVM)\nfrom sklearn.svm import SVC\n#best gamma\ngamma_list=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]\nscores_list=[]\nbest_pipe=pipe\nfor gamma in gamma_list:\n    print(gamma)\n    pipe= Pipeline([('scaler',StandardScaler()),('clf',SVC(kernel='rbf',gamma=gamma,class_weight='balanced'))])\n    scores = cross_val_score(pipe, X, y, cv=5,scoring=make_scorer(f1_score, average='macro'))\n    if all(scores.mean()>x for x in scores_list):\n        best_pipe=pipe\n    scores_list.append(scores.mean())\nscores_list\n"

In [13]:
df_test=df.reindex(df_testids['Customer ID'])
df_test

,Gender,Age,Married,Number of Dependents,Zip Code,Satisfaction Score,Referred a Friend,Number of Referrals,Tenure in Months,Phone Service,...,Offer E,Cable,DSL,Fiber Optic,Month-to-Month,One Year,Two Year,Bank Withdrawal,Credit Card,Mailed Check
Customer ID,,,,,,,,,,,,,,,,,,,,,
9938-EKRGF,0.000000,73.000000,1.000000,0.000000,30.000000,3.000000,1.000000,1.000000,47.000000,0.988258,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7379-POKDZ,1.000000,64.000000,0.000000,0.000000,4.081633,4.000000,0.000000,0.000000,50.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
0654-HMSHN,0.492739,16.588687,1.026365,1.837048,4.000000,5.000000,1.000000,9.000000,52.000000,1.000000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2045-BMBTJ,0.000000,35.000000,1.000000,3.000000,3.286309,4.302025,1.000000,1.000000,11.000000,1.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
0701-TJSEF,1.000000,67.000000,1.000000,0.000000,5.659453,2.337591,1.000000,1.000000,13.000000,1.312852,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4587-VVTOX,1.000000,49.000000,0.000000,0.230715,3.000000,4.000000,0.000000,0.000000,15.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7716-YTYHG,0.000000,78.000000,0.000000,0.000000,3.000000,3.107585,-0.004568,0.017391,31.207662,0.925604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7649-PHJVR,1.000000,27.000000,1.000000,0.000000,6.727180,1.000000,0.937299,1.000000,21.000000,1.000000,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pipe.fit(X,y)
X_test=df_test.to_numpy()
y_test=pipe.predict(X_test)
output=pd.DataFrame(df_testids)
output.insert(1,'Churn Category',y_test)
output.set_index('Customer ID').to_csv('mlp_lbfgs.csv')
# best_pipe.fit(X,y)
# X_test=df_test.to_numpy()
# y_test=best_pipe.predict(X_test)
# output=pd.DataFrame(df_testids)
# output.insert(1,'Churn Category',y_test)
# output.set_index('Customer ID').to_csv('linear_svm.csv')